In [1]:
import pandas as pd
import numpy as np
import os

In [4]:
ccdb_orig_path = "/groups/itay_mayrose/halabikeren/PloiDB/ccdb_all_data_171122.csv"
ccdb_new_nr_path = "/groups/itay_mayrose/halabikeren/CCDB_NR/full_resolved_names.csv"
ppn_nr_path = "/groups/itay_mayrose/halabikeren/plant_pollinator_networks/data/name_resolution/plant_nr_rwfo/processed_resolved_plant_names.csv"

In [5]:
ccdb_orig_nr = pd.read_csv(ccdb_orig_path).drop(["Unnamed: 0", 'parsed_n', 'reference','sporophytic', 'source'], axis=1).drop_duplicates()
ccdb_orig_nr["original_name_with_authority"] = ccdb_orig_nr.original_name
ccdb_orig_nr["sp_name"] = ccdb_orig_nr.original_name.apply(lambda n: " ".join(n.lower().split(" ")[:2]))
ccdb_orig_name_to_ploidb_name = ccdb_orig_nr.set_index("original_name")["mapped_name"].dropna().to_dict()
sp_name_to_ploidb_name = ccdb_orig_nr.set_index("sp_name")["mapped_name"].dropna().to_dict()
ploidb_names = set(ccdb_orig_nr.mapped_name.str.lower())

ccdb_orig_nr["ploidb_name_genus"] = ccdb_orig_nr.mapped_name.apply(lambda n: n.lower().split(" ")[0] if pd.notna(n) else np.nan)
ccdb_orig_nr["original_name_genus"] = ccdb_orig_nr.sp_name.apply(lambda n: n.lower().split(" ")[0])
ccdb_orig_nr["resolved_name_genus"] = ccdb_orig_nr.resolved_name.apply(lambda n: n.lower().split(" ")[0] if pd.notna(n) else np.nan)
ploidb_names_with_syn = ccdb_orig_nr.loc[(ccdb_orig_nr.ploidb_name_genus != ccdb_orig_nr.original_name_genus) & (ccdb_orig_nr.ploidb_name_genus != ccdb_orig_nr.resolved_name_genus)][["original_name", "matched_name", "resolved_name", "sp_name", "mapped_name"]].mapped_name.unique().tolist()

In [6]:
ccdb_new_nr = pd.read_csv(ccdb_new_nr_path, encoding='latin-1')
ccdb_new_nr["original_name"] = ccdb_new_nr["original_name"].fillna(ccdb_new_nr["name_to_resolve"])
ccdb_new_nr["original_name_with_authority"] = (ccdb_new_nr.original_name + " " + ccdb_new_nr.original_authority)
josef_input = pd.read_csv("resolved_chrome_counts_original_names.csv")

/tmp/ipykernel_17076/2244369173.py:1: DtypeWarning: Columns (13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  ccdb_new_nr = pd.read_csv(ccdb_new_nr_path, encoding='latin-1')


In [7]:
names_in_joseph_output_notin_orig_ccdb = (set(josef_input.original_name.str.lower())-set(ccdb_orig_nr.original_name.str.lower()))
named_in_orig_ccdb_notin_joseph_input = (set(ccdb_orig_nr.original_name.str.lower())-set(josef_input.original_name.str.lower()))
names_in_orig_ccdb_notin_new_ccdb = (set(ccdb_orig_nr.original_name.str.lower())-set(ccdb_new_nr.original_name.str.lower()))
print(f"# names that appear in the file josef sent for NR but not in ccdb from 17.11.22 = {len(names_in_joseph_output_notin_orig_ccdb):,}")
print(f"# names that appear in the ccdb from 17.11.22 but not file josef sent for NR = {len(named_in_orig_ccdb_notin_joseph_input):,}")
print(f"# names that appear in the ccdb from 17.11.22 but not in the new CCDB NR = {len(names_in_orig_ccdb_notin_new_ccdb):,}")

# names that appear in the file josef sent for NR but not in ccdb from 17.11.22 = 14,961
# names that appear in the ccdb from 17.11.22 but not file josef sent for NR = 38,601
# names that appear in the ccdb from 17.11.22 but not in the new CCDB NR = 4,763


In [8]:
ccdb_new_nr = ccdb_new_nr.loc[ccdb_new_nr.original_name.notna()]
ccdb_new_nr.original_name = ccdb_new_nr.original_name.str.lower() 
ccdb_new_nr.resolved_name = ccdb_new_nr.resolved_name.str.lower() 
ccdb_new_nr["sp_name"] = ccdb_new_nr.original_name.apply(lambda n: " ".join(n.lower().split(" ")[:2]))
ccdb_new_nr["ploidb_name"] = ccdb_new_nr.sp_name.apply(lambda n: sp_name_to_ploidb_name.get(n, np.nan))
ccdb_new_nr["ploidb_name"] = ccdb_new_nr.apply(lambda rec: rec.ploidb_name if pd.notna(rec.ploidb_name) else ccdb_orig_name_to_ploidb_name.get(rec.original_name, np.nan), axis=1)
# complement using resolved names
ccdb_new_nr["ploidb_name"] = ccdb_new_nr["ploidb_name"].str.lower()
resolved_to_ploidb_name = ccdb_new_nr.dropna(subset=["resolved_name", "ploidb_name"]).set_index("resolved_name").ploidb_name.to_dict()

ccdb_new_nr["ploidb_name_genus"] = ccdb_new_nr.ploidb_name.apply(lambda n: n.split(" ")[0] if pd.notna(n) else np.nan)
ccdb_new_nr["original_name_genus"] = ccdb_new_nr.sp_name.apply(lambda n: n.split(" ")[0])
ccdb_new_nr["resolved_name_genus"] = ccdb_new_nr.resolved_name.apply(lambda n: n.split(" ")[0] if pd.notna(n) else np.nan)

ccdb_new_nr.to_csv(ccdb_new_nr_path, index=False)
print(f"% names in new NR that have no mapped name in the original ccdb = {np.round(ccdb_new_nr.ploidb_name.isna().sum() / ccdb_new_nr.shape[0] * 100, 2)}% ({ccdb_new_nr.ploidb_name.isna().sum():,})")

% names in new NR that have no mapped name in the original ccdb = 0.41% (2,777)


In [9]:
inconsistent_names = ccdb_new_nr.loc[(ccdb_new_nr.ploidb_name_genus != ccdb_new_nr.original_name_genus) & (ccdb_new_nr.ploidb_name_genus != ccdb_new_nr.resolved_name_genus) & (~ccdb_new_nr.ploidb_name.isin(ploidb_names_with_syn)) & (ccdb_new_nr.status != 'Synonym')][["original_name", "matched_name", "resolved_name", "status", "sp_name", "ploidb_name"]].drop_duplicates()
print(f"# names in ploidb that neither match the original nor the resolved names = {len(inconsistent_names.ploidb_name.unique()):,}")
inconsistent_names.groupby("status").ploidb_name.count()

# names in ploidb that neither match the original nor the resolved names = 1,740


status
Accepted     3929
Unchecked     880
Name: ploidb_name, dtype: int64

In [24]:
inconsistent_names.query("status == 'Accepted'").sort_values("ploidb_name").drop_duplicates().head()

,original_name,matched_name,resolved_name,status,sp_name,ploidb_name
136707,zabelia integrifolia (koidz.) makino ex ikuse ...,Zabelia integrifolia,zabelia integrifolia,Accepted,zabelia integrifolia,abelia integrifolia
241452,zabelia integrifolia (koidz.) makino ex ikuse ...,NaN,zabelia integrifolia,Accepted,zabelia integrifolia,abelia integrifolia
136709,zabelia tyaihyonii (nakai) hisauti & h. hara,Zabelia tyaihyoni,zabelia tyaihyoni,Accepted,zabelia tyaihyonii,abelia tyaihyoni
237966,zabelia tyaihyonii (nakai) hisauti & h. hara,NaN,zabelia tyaihyoni,Accepted,zabelia tyaihyonii,abelia tyaihyoni
13275,thuidium abietinum,Thuidium abietinum,thuidium abietinum,Accepted,thuidium abietinum,abietinella abietina var. hystricosa


In [25]:
ccdb_orig_nr.loc[ccdb_orig_nr.mapped_name.isin(inconsistent_names.loc[inconsistent_names.status == 'Accepted', "ploidb_name"])][["original_name", "matched_name", "resolved_name", "sp_name", "mapped_name"]].sort_values("mapped_name").head()

,original_name,matched_name,resolved_name,sp_name,mapped_name
209570,Zabelia integrifolia (Koidz.) Makino ex Ikuse ...,Zabelia integrifolia (Koidz.) Makino ex Ikuse ...,Abelia integrifolia Koidz.,zabelia integrifolia,abelia integrifolia
209572,Zabelia tyaihyonii (Nakai) Hisauti & H. Hara,Zabelia tyaihyonii (Nakai) Hisauti & H. Hara,Abelia tyaihyoni Nakai,zabelia tyaihyonii,abelia tyaihyoni
201654,Acanthophippium bicolor Lindl.,Acanthephippium bicolor Lindl.,Acanthephippium bicolor Lindl.,acanthophippium bicolor,acanthephippium bicolor
271152,Acanthephippium bicolor Lindl.,Acanthephippium bicolor Lindl.,Acanthephippium bicolor Lindl.,acanthephippium bicolor,acanthephippium bicolor
98469,Acanthephippium pictum,Acanthephippium pictum Fukuy.,Acanthephippium pictum Fukuy.,acanthephippium pictum,acanthephippium pictum


In [15]:
ccdb_new_nr.original_name = ccdb_new_nr.original_name.str.lower()
ccdb_new_nr.resolved_name = ccdb_new_nr.resolved_name.str.lower()
original_to_ploidb_name = ccdb_new_nr.dropna(subset=["original_name", "ploidb_name"]).set_index("original_name")["ploidb_name"].to_dict()
resolved_to_ploidb_name = ccdb_new_nr.dropna(subset=["resolved_name", "ploidb_name"]).set_index("resolved_name")["ploidb_name"].to_dict()

In [26]:
ppn_resolved_names = pd.read_csv(ppn_nr_path)
ppn_resolved_names["ploidb_name"] = ppn_resolved_names.resolved_name.apply(lambda n: n if n in ploidb_names else np.nan)
ppn_resolved_names["ploidb_name"] = ppn_resolved_names.apply(lambda rec: rec.ploidb_name if pd.notna(rec.ploidb_name) else resolved_to_ploidb_name.get(rec.resolved_name, np.nan), axis=1)
ppn_resolved_names["ploidb_name"] = ppn_resolved_names.apply(lambda rec: rec.ploidb_name if pd.notna(rec.ploidb_name) else original_to_ploidb_name.get(rec.original_name, np.nan), axis=1)
ppn_resolved_names["ploidb_name"] = ppn_resolved_names.apply(lambda rec: rec.ploidb_name if pd.notna(rec.ploidb_name) else (rec.original_name if rec.original_name in ploidb_names else np.nan), axis=1)
ppn_resolved_names.ploidb_name = ppn_resolved_names.ploidb_name.apply(lambda n: n.lower() if pd.notna(n) else n)
ppn_resolved_names.to_csv(ppn_nr_path, index=False)

In [27]:
ppn_nr_path

'/groups/itay_mayrose/halabikeren/plant_pollinator_networks/data/name_resolution/resolved_plant_names.csv'

In [28]:
ppn_resolved_names.dropna(subset=["resolved_name"]).shape[0]

3964

In [29]:
ppn_resolved_names.dropna(subset=["ploidb_name"]).shape[0]

2418

In [19]:
ppn_resolved_names.loc[(ppn_resolved_names.ploidb_name != ppn_resolved_names.resolved_name) & (~ppn_resolved_names.original_name.str.contains(" "))].dropna(subset=["ploidb_name"])

,original_name,matched_name,resolved_name,ploidb_name
123,lilium,lilium l.,lilium,milium effusum
211,osteospermum,osteospermum l.,osteospermum,monoculus monstrosus
244,lycium,lycium l.,lycium,grabowskia boerhaaviifolia
714,campanula,campanula l.,campanula,campanula myrtifolia
815,sedum,sedum l.,sedum,parvisedum congdonii
1699,colchicum,colchicum l.,colchicum,androcymbium burchellii subsp. pulchrum
1897,utricularia,utricularia l.,utricularia,utricularia australis
2123,mimosa,mimosa l.,mimosa,mimosa quadrivalvis var. occidentalis
2331,thunbergia,thunbergia retz.,thunbergia,thunbergia grandiflora
2507,genista,genista l.,genista,genista monspessulana


In [20]:
inconsistent_ppn_names = ppn_resolved_names.loc[(ppn_resolved_names.resolved_name != ppn_resolved_names.ploidb_name) & (ppn_resolved_names.original_name != ppn_resolved_names.ploidb_name)].dropna(subset=["ploidb_name"]).sort_values("ploidb_name")
inconsistent_ppn_names.head(n=10)

,original_name,matched_name,resolved_name,ploidb_name
554,acer ukurunduense,NaN,NaN,acer caudatum subsp. ukurundense
2734,achillea lanulosa,NaN,NaN,achillea millefolium
2068,cimicifuga simplex,NaN,NaN,actaea simplex
3851,cimicifuga yunnanensis,NaN,NaN,actaea yunnanensis
829,eupatorium odoratum,NaN,NaN,ageratina altissima
4028,ampelopsis brevipedunculata,NaN,NaN,ampelopsis glandulosa var. brevipedunculata
4411,amsinckia intermedia,NaN,NaN,amsinckia menziesii var. intermedia
4037,anagallis arvensis,NaN,NaN,anagallis arvensis subsp. foemina
4010,helichrysum bellidioides,helichrysum bellidiastrum none,helichrysum bellidiastrum,anaphalioides bellidioides
1699,colchicum,colchicum l.,colchicum,androcymbium burchellii subsp. pulchrum


In [21]:
ccdb_new_nr.loc[ccdb_new_nr.ploidb_name.isin(inconsistent_ppn_names.ploidb_name)].sort_values("ploidb_name")[["original_name", "matched_name", "resolved_name", "sp_name", "ploidb_name"]].drop_duplicates().head(n=10)

,original_name,matched_name,resolved_name,sp_name,ploidb_name
621143,acer ukurunduense trautv. & c.a. mey.,Acer ukurunduense,acer caudatum subsp. ukurundense,acer ukurunduense,acer caudatum subsp. ukurundense
392640,acer ukurunduense,Acer ukurunduense,NaN,acer ukurunduense,acer caudatum subsp. ukurundense
36775,acer ukurunduense,Acer ukurunduense,acer ukurunduense,acer ukurunduense,acer caudatum subsp. ukurundense
404682,achillea tanacetifolia,Achillea tanacetifolia,achillea millefolium,achillea tanacetifolia,achillea millefolium
177449,achillea millefolium subsp. millefolium as var...,Achillea millefolium,achillea millefolium,achillea millefolium,achillea millefolium
600010,achillea setacea waldst. et kit.,NaN,achillea setacea,achillea setacea,achillea millefolium
182758,achillea millefolium l. s. l.,NaN,achillea millefolium,achillea millefolium,achillea millefolium
405471,achillea millefolium l. s.)at.,NaN,achillea millefolium,achillea millefolium,achillea millefolium
600502,achillea millefolium l. subsp. millefolium,Achillea millefolium subsp. millefolium,achillea millefolium,achillea millefolium,achillea millefolium
397803,achillea pratensis,Achillea pratensis,achillea millefolium,achillea pratensis,achillea millefolium


In [22]:
ccdb_orig_nr.loc[ccdb_orig_nr.mapped_name.isin(inconsistent_ppn_names.ploidb_name)].sort_values("mapped_name")[["original_name", "matched_name", "resolved_name", "sp_name", "mapped_name"]].drop_duplicates().head(n=10)

,original_name,matched_name,resolved_name,sp_name,mapped_name
201279,Acer ukurunduense Trautv. & C.A. Mey.,Acer ukurunduense Trautv. & C. A. Mey.,Acer caudatum subsp. ukurundense (Trautv. & C....,acer ukurunduense,acer caudatum subsp. ukurundense
274492,Acer ukurunduense,Acer ukurunduense Trautv. & C. A. Mey.,Acer caudatum subsp. ukurundense (Trautv. & C....,acer ukurunduense,acer caudatum subsp. ukurundense
110033,Achillea millefolium L. agg. A. pannonica Scheele,Achillea millefolium L.,Achillea millefolium L.,achillea millefolium,achillea millefolium
110013,Achillea millefolium L.,Achillea millefolium L.,Achillea millefolium L.,achillea millefolium,achillea millefolium
339744,Achillea compacta,Achillea compacta Lam.,Achillea millefolium L.,achillea compacta,achillea millefolium
339767,Achillea tanacetifolia,Achillea tanacetifolia Mill.,Achillea millefolium L.,achillea tanacetifolia,achillea millefolium
347368,Achillea millefolium,Achillea millefolium L.,Achillea millefolium L.,achillea millefolium,achillea millefolium
347380,Achillea pratensis,Achillea pratensis Saukel & R. LÃ¤nger,Achillea millefolium L.,achillea pratensis,achillea millefolium
347381,Achillea setacea,Achillea setacea Schwein.,Achillea millefolium L.,achillea setacea,achillea millefolium
109940,Achillea lanulosa Nutt.,Achillea lanulosa Nutt.,Achillea millefolium L.,achillea lanulosa,achillea millefolium


In [23]:
frac_classified = ppn_resolved_names.ploidb_name.notna().sum() / ppn_resolved_names.shape[0]
frac_resolved = ppn_resolved_names.resolved_name.notna().sum() / ppn_resolved_names.shape[0]
print(f"% ploidy classifications across network plants = {np.round(frac_classified*100, 2)}% ({ppn_resolved_names.ploidb_name.notna().sum():,})")
print(f"% resolved names across network plants = {np.round(frac_resolved*100, 2)}% ({ppn_resolved_names.resolved_name.notna().sum():,})")

% ploidy classifications across network plants = 46.23% (2,418)
% resolved names across network plants = 75.79% (3,964)


In [41]:
original_names_wo_ploidb_mapping = ppn_resolved_names.loc[ppn_resolved_names.ploidb_name.isna()].original_name.dropna().unique().tolist()
resolved_names_wo_ploidb_mapping = ppn_resolved_names.loc[ppn_resolved_names.ploidb_name.isna()].resolved_name.dropna().unique().tolist()

In [42]:
ccdb_orig_nr.loc[(ccdb_orig_nr.mapped_name.notna()) & (ccdb_orig_nr.original_name.str.lower().isin(original_names_wo_ploidb_mapping))]

,family,gametophytic,genus,major_group,matched_name,original_name,resolved_name,taxonomic_status,group,mapped_name,original_name_with_authority,sp_name,ploidb_name_genus,original_name_genus,resolved_name_genus


In [43]:
ccdb_orig_nr.loc[(ccdb_orig_nr.mapped_name.notna()) & (ccdb_orig_nr.resolved_name.str.lower().isin(resolved_names_wo_ploidb_mapping))]

,family,gametophytic,genus,major_group,matched_name,original_name,resolved_name,taxonomic_status,group,mapped_name,original_name_with_authority,sp_name,ploidb_name_genus,original_name_genus,resolved_name_genus


In [44]:
ccdb_new_nr.loc[(ccdb_new_nr.ploidb_name.notna()) & (ccdb_new_nr.original_name.str.lower().isin(original_names_wo_ploidb_mapping))]

,original_name,original_authority,family,genus,species,name_to_resolve,matched_name,resolved_name,resolved_authority,resolved_name_full,resolved_score,status,major_group,original_name_with_authority,ploidb_name,sp_name,ploidb_name_genus,original_name_genus,resolved_name_genus


In [45]:
ccdb_new_nr.loc[(ccdb_new_nr.ploidb_name.notna()) & (ccdb_new_nr.resolved_name.str.lower().isin(resolved_names_wo_ploidb_mapping))]

,original_name,original_authority,family,genus,species,name_to_resolve,matched_name,resolved_name,resolved_authority,resolved_name_full,resolved_score,status,major_group,original_name_with_authority,ploidb_name,sp_name,ploidb_name_genus,original_name_genus,resolved_name_genus


In [46]:
ccdb_new_nr.loc[ccdb_new_nr.ploidb_name == 'pouteria venosa']

,original_name,original_authority,family,genus,species,name_to_resolve,matched_name,resolved_name,resolved_authority,resolved_name_full,resolved_score,status,major_group,original_name_with_authority,ploidb_name,sp_name,ploidb_name_genus,original_name_genus,resolved_name_genus
